Inspired by Domain Languages in "The Pragmatic Programmer" book.

SO the code is very simple, human readable and it will execute using lark. This allows my python to write something more complex in a domain language easily understood by non-specialists.

	•	Lark is the parser generator — it reads the grammar and parses code written in your DSL.
	•	Transformer is used to walk through the parsed tree and convert or execute the parsed data.

In [3]:
import lark
print(lark.__version__)

0.12.0


In [12]:
# mini_language.py
from lark import Lark, Transformer

# === DSL GRAMMAR ===
grammar = """
start: statement+
statement: "when" condition "then" action

condition: "temperature" comparator SIGNED_NUMBER
action: "turn_on" device
device: HEATER | FAN

comparator: GT | LT | EQ

GT: ">"
LT: "<"
EQ: "=="

HEATER: "heater"
FAN: "fan"

%import common.SIGNED_NUMBER
%import common.WS
%ignore WS
"""

be careful, in the class there is no explicit condition on temperature to device. the rules is in the DSL code given below. In here it will just do action if the condition of temperature is satisfied.

In [13]:
# === EVALUATOR ===
class RuleExecutor(Transformer):
    def __init__(self, environment):
        self.env = environment #You pass in a dictionary like {"temperature": 10}.

    def statement(self, items):
        condition_func, action_func = items
        if condition_func():
            action_func() #If it returns True, it calls action_func() to perform the action (e.g., print "Turning on fan").

    def condition(self, items):
        comparator, value = items
        temp = self.env["temperature"]
        ops = {
                    "GT": lambda x: temp > x,
                    "LT": lambda x: temp < x,
                    "EQ": lambda x: temp == x
                }
        return lambda: ops[comparator](float(value)) #Returns a function that checks the condition.

    def comparator(self, items):
        return items[0].type  # "GT", "LT", or "EQ"

    def action(self, items):
        device = items[0]
        return lambda: print(f"[ACTION] Turning on {device}")

    def device(self, items):
        return items[0].type.lower()  # Returns "fan" or "heater" #Extracts the name of the device as a string ("fan" or "heater").

# === USAGE ===
if __name__ == "__main__":
    dsl_code = """
    when temperature > 25 then turn_on fan
    when temperature < 15 then turn_on heater
    """
    print("dsl_code", dsl_code)
    
    parser = Lark(grammar, parser="lalr")
    tree = parser.parse(dsl_code)

    print("== Environment: temperature = 10 ==")
    RuleExecutor({"temperature": 10}).transform(tree)

    print("\n== Environment: temperature = 30 ==")
    RuleExecutor({"temperature": 30}).transform(tree)

dsl_code 
    when temperature > 25 then turn_on fan
    when temperature < 15 then turn_on heater
    
== Environment: temperature = 10 ==
[ACTION] Turning on heater

== Environment: temperature = 30 ==
[ACTION] Turning on fan
